In [1]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import numpy as np
import pandas as pd
import re
import copy
from powergenome.generators import GeneratorClusters
from powergenome.GenX import reduce_time_domain, add_misc_gen_values
from powergenome.params import DATA_PATHS

from powergenome.external_data import (
    make_demand_response_profiles,
    make_generator_variability,
)

from powergenome.load_profiles import (
    make_load_curves, 
    add_load_growth, 
    make_final_load_curves, 
    make_distributed_gen_profiles,
)
from powergenome.external_data import make_demand_response_profiles
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings,
    reverse_dict_of_lists,
    remove_feb_29,
    snake_case_str,
    snake_case_col
)

from pathlib import Path

from powergenome.generators import load_ipm_shapefile
from powergenome.GenX import (
    add_cap_res_network,
    add_co2_costs_to_o_m,
    add_misc_gen_values,
    calculate_partial_CES_values,
    check_resource_tags,
    create_policy_req,
    create_regional_cap_res,
    fix_min_power_values,
    hydro_energy_to_power,
    max_cap_req,
    min_cap_req,
    network_line_loss,
    network_max_reinforcement,
    network_reinforcement_cost,
    reduce_time_domain,
    round_col_values,
    set_int_cols,
    set_must_run_generation,
)
from powergenome.transmission import (
    agg_transmission_constraints,
    transmission_line_distance,
)
from powergenome.fuels import fuel_cost_table

from powergenome.run_powergenome_multiple_outputs_cli import main

from powergenome.external_data import (
    insert_user_tx_costs,
    load_user_tx_costs,
    make_generator_variability,
)

pd.options.display.max_columns = 200

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt

# Outstanding Issues:
- Lifetime in all_gens
- Add more battery ratios (just use NREL ones)

# Temporary Things:
### Hardcoding Battery Ratios

Powergenome assumes the capacity expansion model can co-optimize battery power and energy capacities. Currently, Temoa only optimizes for power capacity and, accordingly, defines different battery technologies for different energy-to-capacity ratios.

Here, we pre-define several energy-to-capacity ratios that we will use in the Temoa model.

In [3]:
battery_ratios = [2, 4, 6, 8, 10] # energy:capacity ratios

# Non-Temporary Things

Define the scenario of interest. This should be from the list of scenarios in the CONUS_extra_inputs/conus_scenario_inputs.csv file ("case_id" column).

In [4]:
scenario = 'base_short_tx_15' #
#current_period = 2040
folder = "pg_inputs_26z_wacc_5_no_ramp/settings/"

scenario = 'base_short_current_policies'
folder = "settings"

In [5]:
cwd = Path.cwd()

settings_path = (
    cwd.parent / "Example-Settings" / folder 
)

settings_path = (
    cwd.parent / "Git-Results-Comparison" / "MIP_results_comparison" / "case_settings" / "26-zone" / folder 
)


settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

pudl_engine, pudl_out, pg_engine = init_pudl_connection(
    freq="AS",
    start_year=min(settings.get("data_years")),
    end_year=max(settings.get("data_years")),
)

check_settings(settings, pg_engine)


            The settings parameter named data_years has been changed to eia_data_years. Please correct it in
            your settings file.

            
The parameter value 'no' from column 'ira' in your scenario definitions file is not included in the 'settings_management' dictionary. Settings for case id 'base_thin' will not be modified to reflect this scenario.
The parameter value 'yes' from column 'policies' in your scenario definitions file is not included in the 'settings_management' dictionary. Settings for case id 'base_thin' will not be modified to reflect this scenario.
The parameter value 'yes' from column 'ccs' in your scenario definitions file is not included in the 'settings_management' dictionary. Settings for case id 'base_thin' will not be modified to reflect this scenario.
The parameter value 'none' from column 'carbon_slack' in your scenario definitions file is not included in the 'settings_management' dictionary. Settings for case id 'base_thin_no_cap' will not b

In [6]:
results_directory = str(settings_path) + '/Computed_Data_' + scenario +'/' #+ '_' + str(current_period) + '/'
isExist = os.path.exists(results_directory)
if not isExist:
    os.makedirs(results_directory)

OK, the scenario settings are now properly set up. 

Let's start defining some local variables to help us w/ our translation layer.

In [7]:
all_periods = []
for p in scenario_settings.keys():
    if scenario in scenario_settings[p].keys():
        all_periods.append(p)


In [8]:
#all_periods = [current_period]

In [9]:
start_year = all_periods[0]

## Load Generator and Fuels Data for All Model Years

We'll now proceed with saving the data for existing generators and new generators. 

Four dataframes will be created:
1. `existing_gens` contains the data characterizing all existing generators.
2. `new_gens` contains the data characterizing new resource options.
3. `all_gens` contains data characterizing both new and existing generators.
4. `gen_variability` contains time-series data for all generators used to generate hourly capacity factors. 

Although the `all_gens` dataframe may be redundant, it is needed as an input to the function that generates the variability data.

We output these four dataframes for every model year and save them as a dictionary, e.g., `new_gens` is a dictionary with keys corresponding to each model year, and the respective values are the dataframes described above.

In [10]:
def calculate_annuity(wacc, n_periods, principal):
    wacc = float(wacc)
    n_periods = int(n_periods)
    principal = float(principal)
    numerator = wacc * principal
    denominator = 1-pow((1+wacc),(-n_periods))
    return abs(numerator / denominator)

def calculate_capex(wacc, n_periods, annuity):
    numerator = annuity * (1 - pow((1+wacc),-n_periods))
    denominator = wacc
    return abs(numerator / denominator)

def determine_capex(df):
    ### There are 2 parts of determining the capex:
    # 1. multiply the stated capex by the regional cost multiplier
    # 2. to that value, the interconnection cost must be added.
    
    for index, row in df.iterrows(): 
        _capex = row.capex_mw * row.regional_cost_multiplier
        # the interconnection cost has different financing terms: 
        # - 60 years
        # - 4.4% WACC
        #wacc = 0.044
        #n_periods = 60
        # Greg has since updated this to 30y at 5%
        wacc = 0.05
        n_periods = 30
        
        tx_capex_orig = row.interconnect_capex_mw
        tx_annuity = calculate_annuity(float(wacc), int(n_periods), float(tx_capex_orig))
        df.loc[index,'Interconnection Capex (test)'] = calculate_capex(wacc, n_periods, tx_annuity)
        tx_capex_new = calculate_capex(row.wacc_real, row.cap_recovery_years, tx_annuity)
        df.loc[index,'Interconnection Capex'] = tx_capex_new
        _capex += tx_capex_new
        df.loc[index,'Total Capex'] = _capex 
        
    return df


In [11]:
new_gens = dict()
all_gens = dict()
existing_gens = dict()
gen_variability = dict()



for year in all_periods:
    print('\n\n Year: ', year)
    _settings = scenario_settings[year][scenario]
    gc = GeneratorClusters(pudl_engine, pudl_out, pg_engine, _settings)
    all_gens[year] = gc.create_all_generators()
    all_gens[year] = add_misc_gen_values(all_gens[year],_settings)
    # Drop columns that were causing csv parsing issues
    if 'plant_id_eia' in all_gens[year].columns:
        all_gens[year].drop(columns = ['plant_id_eia'], axis=1, inplace=True)
    if 'unit_id_pg' in all_gens[year].columns:
        all_gens[year].drop(columns = ['unit_id_pg'], axis=1, inplace=True)
    # Add interconnection costs
    all_gens[year]['Interconnection Capex (test)'] = 0
    all_gens[year]['Interconnection Capex'] = 0
    all_gens[year]['Total Capex'] = 0
    all_gens[year] = determine_capex(all_gens[year])
    all_gens[year].loc[:,'operating_year'] = int(year)
    # end
    if 'Existing_Cap_MW' in all_gens[year].columns:
        existing_gens[year] = all_gens[year][all_gens[year].Existing_Cap_MW > 0]
        new_gens[year] = all_gens[year][all_gens[year].Existing_Cap_MW.isnull()]
    else:
        existing_gens[year] = all_gens[year].head(0)
        new_gens[year] = copy.copy(all_gens[year])
    new_gens[year].loc[:,'operating_year'] = int(year)
    
    gen_variability[year] = make_generator_variability(all_gens[year])
    gen_variability[year].columns = all_gens[year].Resource
    # This normalizes all values in the small_hydroelectric columns
    # gen_variability[year].update(gen_variability[year].filter(like='small_hydroelectric').apply(lambda x: x / x.max()))
    # This sets the values of all small_hydroelectric columns to 1.0
    gen_variability[year].loc[:, gen_variability[year].columns.str.contains('small_hydroelectric')] = 1.0

    if year == all_periods[0]:
        all_gens_for_fuels = all_gens[year][['Fuel', 'region']]
        all_gens_for_fuels = all_gens_for_fuels[all_gens_for_fuels.Fuel != 'None']
        fuels = fuel_cost_table(gc.fuel_prices, generators=all_gens[year], settings=_settings)
        elec_demand = make_final_load_curves(pg_engine, _settings)
        elec_demand.loc[:,'periods'] = all_periods[0]
        
    load_curves_year = make_final_load_curves(pg_engine, _settings)
    load_curves_year.loc[:,'periods']=year
    elec_demand = pd.concat([elec_demand, load_curves_year])
    
    _agff = all_gens[year][['Fuel', 'region']]
    _agff = _agff[_agff.Fuel != 'None']
    all_gens_for_fuels = pd.concat([all_gens_for_fuels, _agff])    
    all_gens_for_fuels.reset_index(inplace=True, drop=True)

    _fuels = fuel_cost_table(gc.fuel_prices, generators=all_gens_for_fuels, settings=_settings)
    _fuels_price = _fuels.iloc[1:,:]
    _fuels_price['year'] = year
    _fuels_co2 = _fuels.iloc[0,:]
    
    if year == all_periods[0]:
        fuels_price = _fuels_price
        fuels_co2 = _fuels_co2
        
    fuels_price = pd.concat([fuels_price, _fuels_price])
    fuels_price.reset_index(inplace=True, drop=True)

fuels_co2.name = 'CO2_Intensity_tons_per_MMBtu'




 Year:  2030
210.70000000000002  MW without lat/lon


********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyCoal Integrated Gasification Combined Cycle changed capacity from 779.0 to 965.5
********************************
********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyConventional Hydroelectric changed capacity from 77722.6 to 77913.2
********************************
********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyNatural Gas Fired Combined Cycle changed capacity from 284907.7 to 288264.5
********************************
********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyNatural Gas Fired Combustion Turbine changed capacity from 138698.5 to 141214.1
********************************
***

Creating gdf
['Solar Photovoltaic', 'Nuclear']
Creating gdf
[]


No distributed generation for the regions {'DC'} was found in your distributed generation data for the scenario MidCase. This is an informational warning -- check your data file if you think there should be distributed generation data in this/these region(s).
No model tag values found for CapRes_1 ('CapRes_1')
No model tag values found for CapRes_2 ('CapRes_2')
No model tag values found for CapRes_3 ('CapRes_3')
No model tag values found for CapRes_4 ('CapRes_4')
No model tag values found for CapRes_5 ('CapRes_5')
No model tag values found for CapRes_6 ('CapRes_6')
No model tag values found for CapRes_7 ('CapRes_7')
No model tag values found for CapRes_8 ('CapRes_8')
No model tag values found for CapRes_9 ('CapRes_9')
No model tag values found for CapRes_10 ('CapRes_10')
No model tag values found for ESR_1 ('ESR_1')
No model tag values found for ESR_2 ('ESR_2')
No model tag values found for ESR_3 ('ESR_3')
No model tag values found for ESR_4 ('ESR_4')
No model tag values found for ESR_



 Year:  2040
210.70000000000002  MW without lat/lon


...

No distributed generation for the regions {'DC'} was found in your distributed generation data for the scenario MidCase. This is an informational warning -- check your data file if you think there should be distributed generation data in this/these region(s).
No model tag values found for CapRes_1 ('CapRes_1')
No model tag values found for CapRes_2 ('CapRes_2')
No model tag values found for CapRes_3 ('CapRes_3')
No model tag values found for CapRes_4 ('CapRes_4')
No model tag values found for CapRes_5 ('CapRes_5')
No model tag values found for CapRes_6 ('CapRes_6')
No model tag values found for CapRes_7 ('CapRes_7')
No model tag values found for CapRes_8 ('CapRes_8')
No model tag values found for CapRes_9 ('CapRes_9')
No model tag values found for CapRes_10 ('CapRes_10')
No model tag values found for ESR_1 ('ESR_1')
No model tag values found for ESR_2 ('ESR_2')
No model tag values found for ESR_3 ('ESR_3')
No model tag values found for ESR_4 ('ESR_4')
No model tag values found for ESR_



 Year:  2050
210.70000000000002  MW without lat/lon


********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyCoal Integrated Gasification Combined Cycle changed capacity from 779.0 to 965.5
********************************
********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyConventional Hydroelectric changed capacity from 77722.6 to 77913.2
********************************
********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyNatural Gas Fired Combined Cycle changed capacity from 284907.7 to 288264.5
********************************
********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyNatural Gas Fired Combustion Turbine changed capacity from 138698.5 to 141214.1
********************************
***

Creating gdf
['Solar Photovoltaic', 'Nuclear']
Creating gdf
[]


No distributed generation for the regions {'DC'} was found in your distributed generation data for the scenario MidCase. This is an informational warning -- check your data file if you think there should be distributed generation data in this/these region(s).
No model tag values found for CapRes_1 ('CapRes_1')
No model tag values found for CapRes_2 ('CapRes_2')
No model tag values found for CapRes_3 ('CapRes_3')
No model tag values found for CapRes_4 ('CapRes_4')
No model tag values found for CapRes_5 ('CapRes_5')
No model tag values found for CapRes_6 ('CapRes_6')
No model tag values found for CapRes_7 ('CapRes_7')
No model tag values found for CapRes_8 ('CapRes_8')
No model tag values found for CapRes_9 ('CapRes_9')
No model tag values found for CapRes_10 ('CapRes_10')
No model tag values found for ESR_1 ('ESR_1')
No model tag values found for ESR_2 ('ESR_2')
No model tag values found for ESR_3 ('ESR_3')
No model tag values found for ESR_4 ('ESR_4')
No model tag values found for ESR_

In [12]:
year = []
daygroup = []
hour = []
timestep = []
dayweight = []
segfrac = []

representative_periods = dict()
reduced_resource_profile = dict()
reduced_load_profile = dict()
load_curves = dict()

## We need to construct the load_curves for all years...
regions = list(all_gens[start_year].region.unique())
for y in all_periods:
    _settings = scenario_settings[y][scenario]
    print('Year: ', y)
    load_curves[y] = make_final_load_curves(pg_engine, _settings)
    
    (
        reduced_resource_profile[y],
        reduced_load_profile[y],
        _time_series_mapping,
        _representative_point,
    ) = reduce_time_domain(gen_variability[y], load_curves[y], _settings)
    
    representative_periods[y] = _representative_point
    
    if _representative_point is None: # 8760 mode
        print('8760 Mode...')
        n_rp = 1
        n_ts_per_rp = 8760
        n_ts = 8760
        year += [y]*n_ts
        _dw = [365]*n_ts
        daygroup += ['8760_mode']*n_ts
        hour += list(np.arange(n_ts_per_rp))
        timestep += list(np.arange(n_ts_per_rp))
        segfrac += list(np.asarray(_dw) / np.asarray(_dw).sum())
        dayweight += _dw
        
        reduced_load_profile[y] = reduced_load_profile[y][regions] # remove all the weird columns.
        reduced_load_profile[y]['year'] = y
        if y == all_periods[0]:
            dfl = copy.copy(reduced_load_profile[y])
        else:
            dfl = pd.concat([dfl, reduced_load_profile[y]])

        
        reduced_resource_profile[y]['year'] = y
        if y == all_periods[0]:
            dfr = copy.copy(reduced_resource_profile[y])
        else:
            dfr = pd.concat([dfr, reduced_resource_profile[y]])
        
        
    else:
        n_rp = int(reduced_load_profile[y].Rep_Periods.values[0])
        n_ts_per_rp = int(reduced_load_profile[y].Timesteps_per_Rep_Period.values[0])
        n_ts = n_rp * n_ts_per_rp
        time_season = _representative_point.slot.values
    
        year += [y]*n_ts
        _dw = []
        for idx, rp in enumerate(time_season):
            daygroup += [rp] *  n_ts_per_rp
            _dw += [reduced_load_profile[y].Sub_Weights.values[idx]] * n_ts_per_rp
            hour += list(np.arange(n_ts_per_rp))
            if idx == 0:
                start = 0
            else:
                start = timestep[-1] + 1
            timestep += list(np.arange(start, n_ts_per_rp + start))
        segfrac += list(np.asarray(_dw) / np.asarray(_dw).sum())
        dayweight += _dw
        reduced_load_profile[y] = reduced_load_profile[y][regions] # remove all the weird columns.
        reduced_load_profile[y]['year'] = y
        if y == all_periods[0]:
            dfl = copy.copy(reduced_load_profile[y])
        else:
            dfl = pd.concat([dfl, reduced_load_profile[y]])
            
        #reduced_resource_profile[y]['year'] = y
#         if y == all_periods[0]:
#             dfr = copy.copy(reduced_resource_profile[y])
#         else:
#             dfr = pd.concat([dfr, reduced_resource_profile[y]],ignore_index=True)




            *****************************
            Regional load data sources have not been specified. Defaulting to EFS load data.
            Check your settings file, and please specify the preferred source for load data
            (FERC, EFS, USER) either for each region or for the entire system with the setting
            "regional_load_source".
            *****************************
            


Year:  2030



            *****************************
            Regional load data sources have not been specified. Defaulting to EFS load data.
            Check your settings file, and please specify the preferred source for load data
            (FERC, EFS, USER) either for each region or for the entire system with the setting
            "regional_load_source".
            *****************************
            


Year:  2040



            *****************************
            Regional load data sources have not been specified. Defaulting to EFS load data.
            Check your settings file, and please specify the preferred source for load data
            (FERC, EFS, USER) either for each region or for the entire system with the setting
            "regional_load_source".
            *****************************
            


Year:  2050


In [13]:
dfl

,BASN,CANO,CASO,FRCC,ISNE,MISC,MISE,MISS,MISW,NWPP,NYCW,NYUP,PJMC,PJMD,PJME,PJMW,RMRG,SPPC,SPPN,SPPS,SRCA,SRCE,SRSE,SRSG,TRE,TRE_WEST,year
0,13589.0,12456.0,19215.0,19720.0,16242.0,21006.0,10336.0,16950.0,24572.0,27422.0,8771.0,7611.0,12621.0,10287.0,28985.0,32350.0,11611.0,8334.0,5324.0,12133.0,18403.0,21184.0,21337.0,14720.0,39674.0,1763.0,2030
1,12855.0,11588.0,17870.0,19075.0,15386.0,20623.0,9948.0,16019.0,23492.0,25650.0,8334.0,7233.0,12158.0,9984.0,28041.0,31714.0,10966.0,8092.0,5094.0,11465.0,18008.0,20912.0,20624.0,13923.0,37764.0,1678.0,2030
2,12277.0,10805.0,16667.0,18937.0,15131.0,20465.0,9758.0,15433.0,22801.0,23853.0,8093.0,6989.0,11977.0,9922.0,27814.0,31540.0,10514.0,7969.0,4956.0,11064.0,17986.0,20928.0,20420.0,13455.0,36660.0,1629.0,2030
3,11836.0,10163.0,15680.0,19262.0,14938.0,20500.0,9713.0,15334.0,22722.0,22659.0,7951.0,6830.0,11913.0,9934.0,27948.0,31674.0,10343.0,7965.0,4929.0,10909.0,18252.0,21201.0,20531.0,13136.0,36288.0,1612.0,2030
4,11687.0,9799.0,15119.0,20028.0,14994.0,20788.0,9682.0,15304.0,22719.0,21955.0,7881.0,6756.0,11914.0,10076.0,28290.0,32076.0,10539.0,8012.0,4936.0,10833.0,18788.0,21614.0,20917.0,13049.0,36315.0,1614.0,2030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,41303.0,32593.0,50398.0,68529.0,26595.0,49437.0,24374.0,44731.0,58195.0,55511.0,16695.0,13600.0,28042.0,32049.0,73015.0,78399.0,26570.0,22131.0,15077.0,33404.0,63859.0,58159.0,70338.0,45950.0,103174.0,4585.0,2050
1676,40010.0,31706.0,49031.0,64053.0,25240.0,47374.0,23044.0,44467.0,55983.0,54116.0,16141.0,13243.0,25980.0,30664.0,68427.0,73934.0,27816.0,21541.0,14808.0,33028.0,58618.0,55956.0,68173.0,45303.0,102216.0,4542.0,2050
1677,40950.0,32614.0,50443.0,57154.0,24437.0,44761.0,21368.0,42549.0,52049.0,56449.0,15885.0,13113.0,24380.0,29251.0,66381.0,70646.0,27252.0,20450.0,14099.0,31621.0,52614.0,51791.0,62007.0,45366.0,97997.0,4355.0,2050
1678,39824.0,31730.0,49077.0,50626.0,23466.0,41449.0,19046.0,39624.0,47145.0,55063.0,14958.0,12456.0,22860.0,26411.0,61138.0,64931.0,25343.0,18885.0,13211.0,29364.0,46447.0,46537.0,54965.0,43816.0,90515.0,4022.0,2050


In [14]:
df_time = pd.DataFrame(
    {'year':year,
    't_season':daygroup,
    't_day':hour,
    'timestep':timestep,
    'dayweight':dayweight,
    'SegFrac':segfrac}
)


In [15]:
dfl.to_csv(results_directory + 'elec_demand_all_years.csv', index=False)
df_time.to_csv(results_directory + 'time.csv', index=False)
for y in reduced_resource_profile.keys():
    reduced_resource_profile[y].to_csv(results_directory + 'capacity_factors_'+str(y)+'.csv', index=False)


In [16]:
model_regions_gdf = gc.model_regions_gdf
if _settings.get("user_transmission_costs"):
    user_tx_costs = load_user_tx_costs(
        _settings["input_folder"]
        / _settings["user_transmission_costs"],
        _settings["model_regions"],
        _settings.get("target_usd_year"),
    )
    transmission = agg_transmission_constraints(
        pg_engine=pg_engine, settings=_settings
    ).pipe(insert_user_tx_costs, user_costs=user_tx_costs)
else:
    transmission = (
        agg_transmission_constraints(
            pg_engine=pg_engine, settings=_settings
        )
        .pipe(
            transmission_line_distance,
            ipm_shapefile=model_regions_gdf,
            settings=_settings,
            units="mile",
        )
        .pipe(network_line_loss, settings=_settings)
        .pipe(network_reinforcement_cost, settings=_settings)
    )
transmission = (
    transmission.pipe(network_max_reinforcement, settings=_settings)
    .pipe(set_int_cols)
    .pipe(round_col_values)
    .pipe(add_cap_res_network, settings=_settings)
)
zones = _settings["model_regions"]
network_zones = [f"z{n+1}" for n in range(len(zones))]
nz_df = pd.Series(data=network_zones, name="Network_zones")
network = pd.concat([pd.DataFrame(nz_df), transmission], axis=1)
network['WACC'] = settings['transmission_investment_cost']['tx']['wacc']
network['investment_years'] = settings['transmission_investment_cost']['tx']['investment_years']

No transmission value column (e.g. firm vs non-firm) was specified in the settings. The column 'firm_ttc_mw' will be used as a default. This is a change from previous versions of PG, where 'nonfirm_ttc_mw' was used. Firm transmission capacity is lower or equal to non-firm capacity.
The user transmission capacity table duplicates values from the database. Database values will be discarded in favor of user values.


In [17]:
network.to_csv(results_directory + 'transmission.csv', index=False)

## Transmission

In [18]:


# if len(settings['region_aggregations'])>1:
#     transmission = agg_transmission_constraints(pg_engine=pg_engine, settings=settings)
#     model_regions_gdf = load_ipm_shapefile(settings)
#     transmission = transmission_line_distance(
#         trans_constraints_df=transmission,
#         ipm_shapefile=model_regions_gdf,
#         settings=settings,
#     )
#     transmission = network_line_loss(transmission=transmission, settings=settings)
#     transmission = network_reinforcement_cost(transmission=transmission, settings=settings)
#     transmission = network_max_reinforcement(transmission=transmission, settings=settings)
#     transmission = add_cap_res_network(transmission, settings)
#     transmission['source'] = transmission.apply(lambda row: row.transmission_path_name.split('_')[0], axis=1)
#     transmission['destination'] = transmission.apply(lambda row: row.transmission_path_name.split('_')[2], axis=1)
    
# transmission.to_csv(results_directory + 'transmission.csv', index=False)


### Let's make some modifications for Temoa

Let's start by stripping the region name off of the tech name.


In [19]:
# for y in new_gens.keys():
#     new_gens[y]['Resource'] = new_gens[y]['Resource'].apply(lambda x: re.sub(r'^.*?_', '', x))
#     if not existing_gens[y].empty:
#         existing_gens[y]['Resource'] = existing_gens[y]['Resource'].apply(lambda x: re.sub(r'^.*?_', '', x))
#     all_gens[y]['Resource'] = all_gens[y]['Resource'].apply(lambda x: re.sub(r'^.*?_', '', x))

Now let's include technology lifetimes and retirement dates.


In [20]:
lifetime_dict = scenario_settings[start_year][scenario]['retirement_ages']
DEFAULT_LIFETIME = 100
DEFAULT_EOL = 2100

In [21]:
for y in new_gens.keys():
    new_gens[y]['lifetime_yrs'] = DEFAULT_LIFETIME
    new_gens[y]['end_of_life'] = DEFAULT_EOL
    existing_gens[y]['lifetime_yrs'] = DEFAULT_LIFETIME
    existing_gens[y]['end_of_life'] = DEFAULT_EOL
    all_gens[y]['lifetime_yrs'] = DEFAULT_LIFETIME
    all_gens[y]['end_of_life'] = DEFAULT_EOL

In [22]:
for y in new_gens.keys():
    for t, lt in lifetime_dict.items():
        snake_name = snake_case_str(t)
        resources = all_gens[y].Resource.str.contains(snake_name)
        if resources.sum() > 0:
            all_gens[y].loc[resources, 'lifetime_yrs'] = lt
            all_gens[y].loc[resources, 'end_of_life'] = lt + all_gens[y].loc[resources, 'operating_year']
    # We need to set the existing tech lifetimes to 100. Retirements are dealt with via the MaxCapacity constraint
    all_gens[y].loc[all_gens[y]['Existing_Cap_MW'] > 0, 'lifetime_yrs'] = 100
    all_gens[y].loc[all_gens[y]['Existing_Cap_MW'] > 0, 'end_of_life'] = 2100
    
for y in new_gens.keys():
    for t, lt in lifetime_dict.items():
        snake_name = snake_case_str(t)
        resources = new_gens[y].Resource.str.contains(snake_name)
        if resources.sum() > 0:
            new_gens[y].loc[resources, 'lifetime_yrs'] = lt
            new_gens[y].loc[resources, 'end_of_life'] = lt + new_gens[y].loc[resources, 'operating_year']


for y in new_gens.keys():           
    # We need to set the existing tech lifetimes to 100. Retirements are dealt with via the MaxCapacity constraint
    existing_gens[y].loc[existing_gens[y]['Existing_Cap_MW'] > 0, 'lifetime_yrs'] = 100
    existing_gens[y].loc[existing_gens[y]['Existing_Cap_MW'] > 0, 'end_of_life'] = 2100
            


Now look at special retirement dates provided by EIA... These are under the 'additional_retirements' header in the resources.yml file.

Each entry has the following format: [plant_id_eia, generator_id, planned_retirement_date]

In [23]:
retirement_dict = scenario_settings[start_year][scenario]['additional_retirements']

## Temporary: Create battery techs with fixed energy-to-power ratios

In [24]:
for y in new_gens.keys():
    if not existing_gens[y].empty:
        existing_gens[y]['Storage_Duration'] = 'N/A'
        indices = existing_gens[y]['STOR'] == 1
        existing_gens[y].loc[indices, 'Storage_Duration'] = existing_gens[y][indices].Existing_Cap_MWh / existing_gens[y][indices].Existing_Cap_MW


In [25]:
battery_ratios

[2, 4, 6, 8, 10]

In [26]:
for y in new_gens.keys():
    batt_indices = []
    rows_to_add = []
    ## Need to deal with Pumped Hydro
    ## Below loops should really only be done for NEW resource options...

    new_gens[y]['Storage_Duration'] = 'N/A'


    for idx, row in new_gens[y].iterrows():
        if row.STOR == 0:
            continue
        if idx in new_gens[y].index:
            new_gens[y] = new_gens[y].drop(index=idx)
        for ratio in battery_ratios:
            _row = copy.copy(row)
            _row.Resource = _row.Resource + '_' + str(ratio) + 'hour'
            _row.technology = _row.technology + '_' + str(ratio) + 'hour'
            _row.Storage_Duration = ratio
            _row.capex_mw += _row.capex_mwh * ratio
            _row.Fixed_OM_Cost_per_MWyr += _row.Fixed_OM_Cost_per_MWhyr * ratio
            rows_to_add.append(_row)

    for _r in rows_to_add:
        new_gens[y] = new_gens[y].append(_r, ignore_index = True)

        

In [27]:
regional_cap_res = create_regional_cap_res(_settings)
regional_cap_res

,Network_zones,CapRes_1,CapRes_2,CapRes_3,CapRes_4,CapRes_5,CapRes_6,CapRes_7,CapRes_8,CapRes_9,CapRes_10
BASN,z1,0.000,0.000,0.161,0.00,0.0000,0.000,0.00,0.000,0.00,0.000
CANO,z2,0.169,0.000,0.000,0.00,0.0000,0.000,0.00,0.000,0.00,0.000
CASO,z3,0.169,0.000,0.000,0.00,0.0000,0.000,0.00,0.000,0.00,0.000
FRCC,z4,0.000,0.000,0.000,0.00,0.0000,0.000,0.15,0.000,0.00,0.000
ISNE,z5,0.000,0.000,0.000,0.00,0.0000,0.000,0.00,0.000,0.00,0.143
MISC,z6,0.000,0.000,0.000,0.00,0.0000,0.179,0.00,0.000,0.00,0.000
MISE,z7,0.000,0.000,0.000,0.00,0.0000,0.179,0.00,0.000,0.00,0.000
MISS,z8,0.000,0.000,0.000,0.00,0.0000,0.179,0.00,0.000,0.00,0.000
MISW,z9,0.000,0.000,0.000,0.00,0.0000,0.179,0.00,0.000,0.00,0.000
NWPP,z10,0.000,0.000,0.161,0.00,0.0000,0.000,0.00,0.000,0.00,0.000


In [28]:
# By default, GenX names the regions z1, z2, ....
# We need to change that back to region names in the regional_cap_res dataframe.
regional_cap_res['Network_zones'] = regional_cap_res.index
regional_cap_res

,Network_zones,CapRes_1,CapRes_2,CapRes_3,CapRes_4,CapRes_5,CapRes_6,CapRes_7,CapRes_8,CapRes_9,CapRes_10
BASN,BASN,0.000,0.000,0.161,0.00,0.0000,0.000,0.00,0.000,0.00,0.000
CANO,CANO,0.169,0.000,0.000,0.00,0.0000,0.000,0.00,0.000,0.00,0.000
CASO,CASO,0.169,0.000,0.000,0.00,0.0000,0.000,0.00,0.000,0.00,0.000
FRCC,FRCC,0.000,0.000,0.000,0.00,0.0000,0.000,0.15,0.000,0.00,0.000
ISNE,ISNE,0.000,0.000,0.000,0.00,0.0000,0.000,0.00,0.000,0.00,0.143
MISC,MISC,0.000,0.000,0.000,0.00,0.0000,0.179,0.00,0.000,0.00,0.000
MISE,MISE,0.000,0.000,0.000,0.00,0.0000,0.179,0.00,0.000,0.00,0.000
MISS,MISS,0.000,0.000,0.000,0.00,0.0000,0.179,0.00,0.000,0.00,0.000
MISW,MISW,0.000,0.000,0.000,0.00,0.0000,0.179,0.00,0.000,0.00,0.000
NWPP,NWPP,0.000,0.000,0.161,0.00,0.0000,0.000,0.00,0.000,0.00,0.000


In [29]:
regional_cap_res.to_csv(results_directory + 'regional_cap_res.csv', index=False)
fuels_price.to_csv(results_directory + 'fuels_price.csv', index=False)
fuels_co2.to_frame().to_csv(results_directory + 'fuels_co2.csv', index=True)
dfl.to_csv(results_directory + 'elec_demand_all_years.csv', index=False)
df_time.to_csv(results_directory + 'time.csv', index=False)

for y in new_gens.keys():
    gen_variability[y]['model_year'] = y
    gen_variability[y].to_csv(results_directory + 'gen_variability_'+str(y)+'.csv', index=False)
    load_curves[y]['model_year'] = y
    load_curves[y].to_csv(results_directory + 'load_curves_'+str(y)+'.csv', index=False)
    reduced_load_profile[y].to_csv(results_directory + 'reduced_load_profile_'+str(y)+'.csv', index=False)
    reduced_resource_profile[y]['model_year'] = y
    reduced_resource_profile[y].to_csv(results_directory + 'reduced_resource_'+str(y)+'.csv', index=False)
    all_gens[y]['tech_index'] = all_gens[y].index
    all_gens[y]['model_year'] = y
    all_gens[y].to_csv(results_directory + 'all_gens_'+str(y)+'.csv', index=False)
    new_gens[y]['model_year'] = y
    new_gens[y].to_csv(results_directory + 'new_gens_'+str(y)+'.csv', index=False)
    existing_gens[y]['model_year'] = y
    existing_gens[y].to_csv(results_directory + 'existing_gens_'+str(y)+'.csv', index=False)


In [30]:
for y in all_periods:
    if y == all_periods[0]:
        ag = all_gens[y]
        eg = existing_gens[y]
        ng = new_gens[y]
        rlp = reduced_load_profile[y]
    else:
        ag = pd.concat([ag, all_gens[y]])
        eg = pd.concat([eg, existing_gens[y]])
        ng = pd.concat([ng, new_gens[y]])
        rlp = pd.concat([rlp, reduced_load_profile[y]])

In [31]:
df_policies = pd.read_csv(str(settings.get("input_folder")) + '/' + settings.get("emission_policies_fn"))

In [32]:
ag.to_csv(results_directory + 'all_gens' + '.csv', index=False)
eg.to_csv(results_directory + 'existing_gens' + '.csv', index=False)
ng.to_csv(results_directory + 'new_gens' + '.csv', index=False)
rlp.to_csv(results_directory + 'reduced_load_profile' + '.csv', index=False)
df_policies.to_csv(results_directory + 'policies' + '.csv')

In [33]:
gen_variability[2050].to_csv('gv2050.csv')
load_curves[2050].to_csv('lc2050.csv')

In [34]:
dfl

,BASN,CANO,CASO,FRCC,ISNE,MISC,MISE,MISS,MISW,NWPP,NYCW,NYUP,PJMC,PJMD,PJME,PJMW,RMRG,SPPC,SPPN,SPPS,SRCA,SRCE,SRSE,SRSG,TRE,TRE_WEST,year
0,13589.0,12456.0,19215.0,19720.0,16242.0,21006.0,10336.0,16950.0,24572.0,27422.0,8771.0,7611.0,12621.0,10287.0,28985.0,32350.0,11611.0,8334.0,5324.0,12133.0,18403.0,21184.0,21337.0,14720.0,39674.0,1763.0,2030
1,12855.0,11588.0,17870.0,19075.0,15386.0,20623.0,9948.0,16019.0,23492.0,25650.0,8334.0,7233.0,12158.0,9984.0,28041.0,31714.0,10966.0,8092.0,5094.0,11465.0,18008.0,20912.0,20624.0,13923.0,37764.0,1678.0,2030
2,12277.0,10805.0,16667.0,18937.0,15131.0,20465.0,9758.0,15433.0,22801.0,23853.0,8093.0,6989.0,11977.0,9922.0,27814.0,31540.0,10514.0,7969.0,4956.0,11064.0,17986.0,20928.0,20420.0,13455.0,36660.0,1629.0,2030
3,11836.0,10163.0,15680.0,19262.0,14938.0,20500.0,9713.0,15334.0,22722.0,22659.0,7951.0,6830.0,11913.0,9934.0,27948.0,31674.0,10343.0,7965.0,4929.0,10909.0,18252.0,21201.0,20531.0,13136.0,36288.0,1612.0,2030
4,11687.0,9799.0,15119.0,20028.0,14994.0,20788.0,9682.0,15304.0,22719.0,21955.0,7881.0,6756.0,11914.0,10076.0,28290.0,32076.0,10539.0,8012.0,4936.0,10833.0,18788.0,21614.0,20917.0,13049.0,36315.0,1614.0,2030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,41303.0,32593.0,50398.0,68529.0,26595.0,49437.0,24374.0,44731.0,58195.0,55511.0,16695.0,13600.0,28042.0,32049.0,73015.0,78399.0,26570.0,22131.0,15077.0,33404.0,63859.0,58159.0,70338.0,45950.0,103174.0,4585.0,2050
1676,40010.0,31706.0,49031.0,64053.0,25240.0,47374.0,23044.0,44467.0,55983.0,54116.0,16141.0,13243.0,25980.0,30664.0,68427.0,73934.0,27816.0,21541.0,14808.0,33028.0,58618.0,55956.0,68173.0,45303.0,102216.0,4542.0,2050
1677,40950.0,32614.0,50443.0,57154.0,24437.0,44761.0,21368.0,42549.0,52049.0,56449.0,15885.0,13113.0,24380.0,29251.0,66381.0,70646.0,27252.0,20450.0,14099.0,31621.0,52614.0,51791.0,62007.0,45366.0,97997.0,4355.0,2050
1678,39824.0,31730.0,49077.0,50626.0,23466.0,41449.0,19046.0,39624.0,47145.0,55063.0,14958.0,12456.0,22860.0,26411.0,61138.0,64931.0,25343.0,18885.0,13211.0,29364.0,46447.0,46537.0,54965.0,43816.0,90515.0,4022.0,2050
